# Quantum Krylov

- https://arxiv.org/abs/2109.06868
- https://arxiv.org/abs/2211.16097
- https://arxiv.org/abs/2208.00567

## TODO:

-  !!! Make a new branch !!!
- You need to download inquanto to generate the Hamiltonians
- You to download inquanto.extensions.pyscf to do the classical Hartree fock calculations to do the integrals

- Generate a scipy (matrix multiplication) code to do the do the Quantum Krylov algorithm algorithm. (Hint use scipy.linalg.expm exponentiate matrices)
- Use this for benchmarking the quantum circuit hadamard test version
- You will need to use a generalised eigenvalue problem solver code (provided)

# Generate Hamiltonian
- Equilibrium H2
- Solve ground state problem

In [1]:
from inquanto.extensions.pyscf import ChemistryDriverPySCFMolecularRHF

def get_qubit_hamiltonian(bond_length):
    xyz = [
        ["H", [0., 0., 0.]],
        ["H", [0., bond_length, 0.]],
        #  ["H", [0., 2*bond_length, 0.]],
        # ["H", [0., 3*bond_length, 0.]],
    ]

    ferm_ham, fock_space, fock_state = ChemistryDriverPySCFMolecularRHF(geometry=xyz, basis="sto3g").get_system()
    qubit_ham = ferm_ham.qubit_encode()
    qubit_ham_matrix = qubit_ham.to_sparse_matrix().todense()
    return qubit_ham_matrix

qubit_ham_matrix_735 = get_qubit_hamiltonian(0.735)

## Exact GS
- Perfect input to algorithm for testing

In [2]:
import numpy
e,c = numpy.linalg.eigh(qubit_ham_matrix_735)
gs_735 = c[:,0]

## How to calculate expectation values
- $\langle \psi | \hat{H} | \psi \rangle$

In [3]:
gs_735.conj().T @ qubit_ham_matrix_735 @ gs_735

matrix([[-1.13730604+0.j]])

# You will need your own circuit state preperation step $|\psi_0 \rangle$.
- Intro: Use the unentangled HF state $|11\cdots 00 \cdots \rangle$
- Easy: Use inquanto ansatz after VQE or imaginary time optimisation for this
- Hard: Implement imaginary time with post selection from this paper https://arxiv.org/abs/2304.07917

# Eigensolver
- Here is a generalised eiegn value solver
- It takes in the Hamiltonian $\mathbf{H}$ where you need to calculate the elements $H_{ij}$
- It takes in the Overlap Matrix $\mathbf{S}$ where you need to calculate the elements $S_{ij}$. (In this example the overlap is diagonal because the basis is already orthogonal, this is not the case you this problem)
- The linear dependancies are removed withe
- It returns the energies of all states, the eigenvectors, and the overlap matrix eigenvalues (which is a measure of the linear dependancies. 0 is a linear dependance)

- The aim of the project is to populate $\mathbf{H}$ and $\mathbf{S}$

In [8]:
from generalised_eigensolver import gen_eigh
gen_eigh(qubit_ham_matrix_735, numpy.eye(qubit_ham_matrix_735.shape[0]), lindep=1e-10)

(array([-1.13730604, -0.53637008, -0.53637008, -0.52461556, -0.52461556,
        -0.52461556, -0.44066274, -0.44066274, -0.16275316,  0.24807299,
         0.24807299,  0.36664389,  0.36664389,  0.49505774,  0.71996899,
         0.93424723]),
 array([[ 0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  1.        +0.j,
          0.        +0.j],
        [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  1.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j],
        [ 0.        +0.j,  0.        +0.j,  0.        +0.j,
          0.        +0.j,  0.        +0.j,  